In [1]:
import numpy as np
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

In [2]:
with open('words.txt', 'rb') as f:
    words = set()
    for i, w in enumerate(tqdm(f.readlines())):
        try:
            w = w.decode().strip('\n')
            if w.isalpha(): 
                words.add(w.lower())
        except:
            pass

In [3]:
start_meta = '<s>'
end_meta = '<e>'
mask_meta = '<m>'
letters = 'abcdefghijklmnopqrstuvwxyz'

vocab = [start_meta, end_meta, mask_meta] + list(letters)
vocab_to_ind = {w:i for i,w in enumerate(vocab)}

In [4]:
import models

model = models.pointer_transformer(alphabet_size=len(vocab),
                                   dim=64,
                                   num_heads=6,
                                   blocks=4,
                                   mask_meta=vocab_to_ind[mask_meta])

W1006 09:40:21.759448 4563035584 deprecation_wrapper.py:119] From /Users/mshlis/Projects/ANNagram/models.py:5: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W1006 09:40:21.760257 4563035584 deprecation_wrapper.py:119] From /Users/mshlis/Projects/ANNagram/models.py:5: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.

W1006 09:40:21.806939 4563035584 deprecation.py:506] From /Users/mshlis/anaconda3/envs/tf_1.x/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1006 09:40:21.818880 4563035584 deprecation.py:506] From /Users/mshlis/anaconda3/envs/tf_1.x/lib/python3.7/site-packages/tensorflow/python/ops/in

In [5]:
lengths = [len(word) for word in words]

def translate(word, with_metas):
    res = [vocab_to_ind[c] for c in word]
    if not with_metas:
        return res
    return [vocab_to_ind[start_meta]] + res + [vocab_to_ind[end_meta]]

X0 = []
X1 = []
maxlen = 12

for word in tqdm(words):
    if len(word) < maxlen:
        X0.append(translate(word, with_metas=False))
        X1.append(translate(word, with_metas=True))


fill_value = vocab_to_ind[mask_meta]
def pad_sequences(sequences, maxlen, fill_value):
    X = fill_value * np.ones((len(sequences), maxlen))
    lengths = [len(z) for z in sequences]
    mask = np.arange(maxlen) < np.array(lengths).reshape(-1,1)
    X[mask] = np.concatenate(sequences)
    return X

X0 = pad_sequences(X0, maxlen, vocab_to_ind[mask_meta])
X1 = pad_sequences(X1, maxlen+2, vocab_to_ind[mask_meta])
Y = np.concatenate([X1[:,1:], vocab_to_ind[mask_meta]*np.ones((len(X0), 1))], -1)

print(X0.shape, X1.shape, Y.shape)


(329433, 12) (329433, 14) (329433, 14)


In [8]:
import pickle as pkl
pkl.dump([X0, X1, Y, vocab_to_ind], open('data.npy', 'wb'))

In [11]:
import tensorflow as tf
from tensorflow import keras

def loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true[:, :, vocab_to_ind[mask_meta]], 1.), tf.float32)
    logits = tf.log(y_pred + 1e-9)
    return - tf.expand_dims(mask,-1) * y_true * logits

model.compile('adam', loss)

In [ ]:
_X1 = np.random.choice(len(vocab), size=(2400, 14))
_X0 = np.random.choice(len(vocab), size=(2400, 12))
_Y = np.ones((2400,14,12)) / 12

history = model.fit([_X1, _X0], _Y, batch_size=20, epochs=3)

Epoch 1/3
